In [1461]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    "DRIVER={SQL Server};SERVER=lenovo-desktop;DATABASE=Qihuo;UID=samtsql;PWD=F(W}q:TsyK,7^+>`P28e79s#Uc5n")
query = """select m1.date as 'Date', m1.[Close] as 'Market', m2.[close] as 'Stock'  from MinuteQuote m1
inner join MinuteQuote m2 on m1.Date = m2.date
where m1.Date > '2020-04-08 01:20:00' and m1.Contract = 'ym' and m2.Contract = 'msft'
order by m1.date"""
df = pd.read_sql(query, conn)
df.set_index('Date', inplace=True)


c:\Users\SHSIEH\Documents\ProjectsFuture\ML\project-walkthroughs\stock\stock_env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [1462]:
df["StockReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Stock"]
df["MarketReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Market"]
df["OutPerform"] = df["StockReturn"] - df["MarketReturn"]
df["Target"] = (df.apply(lambda x: x > 0)["OutPerform"]).astype(int)

# df.plot.line(y="OutPerform", use_index=True)

In [1463]:
predictors = []
for i in range(12):
    df[f'OutPerform{pow(2,i)*5}'] = df['OutPerform'].rolling(pow(2,i)).sum()
    predictors.append(f'OutPerform{pow(2,i)*5}')
    df[f'StockReturn{pow(2,i)*5}'] = df['StockReturn'].rolling(pow(2,i)).sum()
    predictors.append(f'StockReturn{pow(2,i)*5}')

In [1464]:
prev = df.copy()
prev = prev.shift(1)
data = df[["OutPerform","Target"]]


In [1465]:
data = data.join(prev[predictors])
data = data.copy().dropna()


In [1466]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import precision_score

model = RandomForestClassifier(n_estimators=100, min_samples_split=1200, random_state=1)


In [1467]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [1468]:
def backtest(data, model, predictors, start=50000, step=1000):
    all_predictions = []
    loop = 0
    for i in range(start, data.shape[0], step):
        train = data.iloc[loop * step:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        loop = 0

    return pd.concat(all_predictions)


In [1469]:
data.shape[0]

40312

In [1470]:
predictions = backtest(data, model, predictors)

In [1471]:
print(predictions)

                     Target  Predictions
Date                                    
2022-05-19 12:55:00       1            1
2022-05-20 06:30:00       0            1
2022-05-20 06:35:00       0            0
2022-05-20 06:40:00       0            1
2022-05-20 06:45:00       0            1
...                     ...          ...
2022-05-25 12:30:00       0            1
2022-05-25 12:35:00       0            1
2022-05-25 12:40:00       0            1
2022-05-25 12:45:00       1            0
2022-05-25 12:50:00       0            0

[312 rows x 2 columns]


In [1472]:
predictions["Predictions"].value_counts()

1    183
0    129
Name: Predictions, dtype: int64

In [1473]:
precision_score(predictions["Target"], predictions["Predictions"])

0.5136612021857924

In [1474]:
predictions["Target"]

Date
2022-05-19 12:55:00    1
2022-05-20 06:30:00    0
2022-05-20 06:35:00    0
2022-05-20 06:40:00    0
2022-05-20 06:45:00    0
                      ..
2022-05-25 12:30:00    0
2022-05-25 12:35:00    0
2022-05-25 12:40:00    0
2022-05-25 12:45:00    1
2022-05-25 12:50:00    0
Name: Target, Length: 312, dtype: int32